In [3]:
import src.config as config
import datetime
import pandas as pd
import hopsworks
from src2.data import load_raw_data



In [20]:
time_now = pd.to_datetime(datetime.datetime.now(datetime.timezone.utc)).floor('h')
fetch_data_from = time_now - datetime.timedelta(days=28)
fetch_data_to = time_now
fetch_data_from, fetch_data_to

(Timestamp('2025-03-24 11:00:00+0000', tz='UTC'),
 Timestamp('2025-04-21 11:00:00+0000', tz='UTC'))

In [10]:
datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(days=28)

datetime.datetime(2025, 3, 24, 10, 53, 15, 810278, tzinfo=datetime.timezone.utc)

In [11]:
def fetch_batch_of_recent_data(from_date, to_date):

    from_date_ = from_date - datetime.timedelta(days=7*52)
    to_date_ = to_date - datetime.timedelta(days=7*52)

    rides_1 = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides_1 = rides_1.loc[
        rides_1['pickup_datetime'] >= from_date_.to_datetime64()]
    
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2.loc[
        rides_2['pickup_datetime'] < to_date_.to_datetime64()]
    
    rides = pd.concat([rides_1,rides_2])

    rides['pickup_datetime'] = rides['pickup_datetime'] + datetime.timedelta(days=7*52)

    return rides
rides = fetch_batch_of_recent_data(fetch_data_from,fetch_data_to)
rides




file already existed on disk for 2024-03
file already existed on disk for 2024-04


,pickup_datetime,location_id
2475892,2025-03-24 22:50:00,114
2487078,2025-03-24 10:02:10,90
2491135,2025-03-24 10:58:45,112
2491136,2025-03-24 10:59:21,112
2491137,2025-03-24 10:41:15,161
...,...,...
3454538,2025-04-21 09:34:30,262
3454539,2025-04-21 09:25:20,236
3454540,2025-04-21 09:09:22,91
3454541,2025-04-21 09:19:06,100


In [12]:
rides.pickup_datetime.value_counts()

2025-04-01 18:15:00    17
2025-04-02 21:41:00    15
2025-04-10 20:03:00    15
2025-04-09 21:48:00    14
2025-03-27 20:27:00    14
                       ..
2025-04-13 00:54:41     1
2025-03-31 11:24:40     1
2025-04-13 00:47:30     1
2025-04-13 00:48:27     1
2025-04-21 09:02:01     1
Name: pickup_datetime, Length: 1576685, dtype: int64

In [45]:
rides[0].loc[rides[0].pickup_datetime < (fetch_data_to.to_datetime64())]['pickup_datetime']

107051    2025-03-01 23:57:11
151710    2025-03-01 14:00:46
152755    2025-03-01 14:03:06
153100    2025-03-01 14:01:18
153268    2025-03-01 14:02:30
                  ...        
3556421   2025-03-29 13:47:29
3556422   2025-03-29 13:14:05
3556423   2025-03-29 13:44:23
3556424   2025-03-29 13:56:30
3556425   2025-03-29 13:25:57
Name: pickup_datetime, Length: 6664697, dtype: datetime64[ns]

In [13]:
from src2.data import transform_raw_into_ts_data

In [14]:
ts_data = transform_raw_into_ts_data(rides)

2025-04-21 06:54:02,058 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

2025-04-21 06:54:02,068 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [9]:
ts_data.loc[ts_data.pickup_hour > '2025-04-01 08:00:00']


,pickup_hour,rides,location_id
670,2025-04-01 09:00:00,12,4
671,2025-04-01 10:00:00,5,4
1342,2025-04-01 09:00:00,5,7
1343,2025-04-01 10:00:00,1,7
2014,2025-04-01 09:00:00,2,10
...,...,...,...
172031,2025-04-01 10:00:00,0,84
172702,2025-04-01 09:00:00,0,187
172703,2025-04-01 10:00:00,0,187
173374,2025-04-01 09:00:00,0,156


In [13]:
pd.to_datetime(datetime.datetime.now(datetime.timezone.utc)).to_datetime64()

numpy.datetime64('2025-03-29T09:25:12.792956000')

In [15]:
HOPSWORK_PROJECT_NAME = config.HOPSWORK_PROJECT_NAME
HOPSWORK_API_KEY = config.HOPSWORK_API_KEY

In [16]:
project = hopsworks.login(project=HOPSWORK_PROJECT_NAME, api_key_value=HOPSWORK_API_KEY)


2025-04-21 06:54:26,811 INFO: Initializing external client
2025-04-21 06:54:26,814 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-21 06:54:29,098 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-21 06:54:41,692 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214618


In [17]:
feature_store = project.get_feature_store()

In [18]:
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version = config.FEATURE_GROUP_VERSION,
    description = 'time series data at hourly frequency',
    primary_key = ['pickup_hour','location_id'],
    event_time = 'pickup_hour'
    )

In [19]:
feature_group.insert(ts_data, write_options={'wait_for_job':False})

Uploading Dataframe: 100.00% |██████████| Rows 172704/172704 | Elapsed Time: 00:27 | Remaining Time: 00:00


Launching job: time_series_data_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214618/jobs/named/time_series_data_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_data_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [19]:
ts_data

,pickup_hour,rides,location_id
0,2025-03-04 11:00:00,1,4
1,2025-03-04 12:00:00,2,4
2,2025-03-04 13:00:00,2,4
3,2025-03-04 14:00:00,3,4
4,2025-03-04 15:00:00,6,4
...,...,...,...
173371,2025-04-01 06:00:00,0,156
173372,2025-04-01 07:00:00,0,156
173373,2025-04-01 08:00:00,0,156
173374,2025-04-01 09:00:00,0,156
